In [40]:
import os
import shutil
import pandas as pd
import numpy as np

This section reads 2 VtV files and prepares them to be merged. method == 0 reads files that end with _single.txt, these are the single speaker transcriptions produced from videos where there is only 1 speaker. method != 0 reads files that end with _single_X.txt and _single_Y.txt, these are single speaker transcriptions produced by splitting the speakers from videos with 2 speakers.

In [47]:
method = 0
if method == 0: # Merge seperate transcriptions
    directory = "Transcripts/Study 2 (Fall2023)/VTV/"
    file_start = "VtV36"
    
    dfA = pd.read_csv(directory + "single/"+ file_start + "A_single.txt", sep="|")
    dfB = pd.read_csv(directory + "single/"+ file_start + "B_single.txt", sep="|")
else: # Merge pre split transcriptions
    directory = "Transcripts/Study 2 (Fall2023)/VTV/"
    file_start = "VtV17"
    
    dfA = pd.read_csv(directory + "single/"+ file_start + "A_single_X.txt", sep="|")
    dfB = pd.read_csv(directory + "single/"+ file_start + "B_single_Y.txt", sep="|")

To merge, we select 2 timestamps in both transcriptions that have been manually determined to be identical in actual time. We then calculate the difference and subtract the difference from the first transcriptions file, which synchronizes the timestamps for both files.

In [48]:
dfA_time = 123.28999999999999
dfB_time = 122.24000000000001
difference = dfA_time - dfB_time

In [49]:
dfA["Start Time"] = dfA["Start Time"] -difference
dfA["End Time"]= dfA["End Time"] -difference
dfA["Speaker"] = "Speaker 1"
dfB["Speaker"] = "Speaker 2"

We can then merge both files and then sort by the start time of each line of speech. This produces a merged transcription that we can now save to a csv file.

In [50]:
df_new = pd.concat([dfA, dfB])
df_new = df_new.sort_values(by=["Start Time"])
df_new.to_csv(directory + "dyad/"+ file_start + "_dyad.txt", index=False,sep="|")